# Different Solutions for 1-Dimensional-1-Phase Problem

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://drive.google.com/file/d/1VsPyLcLGSQwQD8EHkY-UdUPreWSPvy9Y/view?usp=drive_link)

*Author: Zakariya Abugrin | Date: November 2023*

## Introduction

The solution for a simple 1-dimensional-1-phase (1D1P) problem is available in all solutions which includes: `analytical`, `numerical`, `neurical`. In tutorial, we will see how we can build all solutions and compare them together. 

## Prerequisites:
Before you can follow this tutorial, you need to understand the following tutorials:
1. [Understand scalers Module](/user_guide/tutorials/tutorial_understand_scalers_module.html).
2. [Understand Compiling](/user_guide/tutorials/tutorial_understand_compiling.html).

## Import `reservoirflow`

We start with importing `reservoirflow` as `rf`. The abbreviation `rf` refers to `reservoirflow` where all modules under this library can be accessed. `rf` is also used throughout the [documentation](/api/API.html). We recommend our users to stick with this convention.

In [1]:
import reservoirflow as rf
import numpy as np

print(rf.__version__)

0.1.0


## Create a Model

In [2]:
run_config = dict(
    nsteps=100,
    threading=True,
    vectorize=True,
    check_MB=True,
    print_arrays=False,
    isolver=None,  #'cgs',
)

In [3]:
def create_model(nx=101, n_w=2, comp=False):
    if comp:
        g_comp = 1 * 10**-6
        f_comp = 1 * 10**-5
    else:
        g_comp = None
        f_comp = None

    grid = rf.grids.RegularCartesian(
        nx=nx,
        ny=1,
        nz=1,
        dx=300,
        dy=350,
        dz=40,
        phi=0.27,
        kx=270,
        comp=g_comp,
        dtype="double",
    )

    fluid = rf.fluids.SinglePhase(
        mu=0.5,
        B=1,
        comp=f_comp,
        dtype="double",
    )

    model = rf.models.BlackOil(
        grid, fluid, pi=4000, dtype="double", dt=1, verbose=False, sparse=True
    )
    if n_w > 0:
        well_ids = np.cumsum([nx // (n_w + 1) for i in range(n_w)])
        print("Wells_ids:", well_ids)
        for well_id in well_ids:
            # model.set_well(id=well_id, q=-600, s=1.5, r=3.5)
            model.set_well(id=well_id, pwf=1000, s=1.5, r=3.5)
        model.set_boundaries({0: ("pressure", 4000), nx + 1: ("pressure", 4000)})
    else:
        model.set_boundaries({0: ("pressure", 4000), nx + 1: ("pressure", 1000)})
        # model.set_boundaries({0: ("pressure", 1000), nx+1: ("pressure", 4000)})

    return model

In [4]:
model = create_model(n_w=0, comp=True)

In [5]:
model.compile("numerical", "FDM")
model.run(**run_config)

[info] FDM was assigned as model.solution.
[info] Simulation run started: 100 timesteps.


[step] 100: 100%|██████████| 100/100 [00:00<00:00, 187.83steps/s]

[info] Simulation run of 100 steps finished in 0.54 seconds.
[info] Material Balance Error: 7.602807272633072e-13.


In [6]:
model.compile("analytical", "1D1P")

[info] D1P1 was assigned as model.solution.


In [7]:
model.solution.calc_solution()



     ncalls  tottime  percall  cumtime  percall.1  \
0        1   65.029   65.029   65.049     65.049   
1        1    0.011    0.011    0.011      0.011   
2        7    0.006    0.001    0.006      0.001   
3        1    0.002    0.002    0.002      0.002   
4        7    0.000    0.000    0.000      0.000   
..     ...      ...      ...      ...        ...   
62       3    0.000    0.000    0.000      0.000   
63       2    0.000    0.000    0.000      0.000   
64       1    0.000    0.000    0.000      0.000   
65       1    0.000    0.000    0.000      0.000   
66       1    0.000    0.000    0.000      0.000   

                            filename:lineno(function)  
0   C:\Users\abugrzka\Projects\hiesab\reservoirflo...  
1   C:\Users\abugrzka\Projects\hiesab\reservoirflo...  
2          {method 'copy' of 'numpy.ndarray' objects}  
3   c:\Users\abugrzka\Projects\hiesab\reservoirflo...  
4   c:\Users\abugrzka\Projects\hiesab\reservoirflo...  
..                                  

(array([[[0.000e+00, 1.500e+02],
         [0.000e+00, 4.500e+02],
         [0.000e+00, 7.500e+02],
         ...,
         [0.000e+00, 3.015e+04],
         [0.000e+00, 3.045e+04],
         [0.000e+00, 3.075e+04]],
 
        [[1.000e+00, 1.500e+02],
         [1.000e+00, 4.500e+02],
         [1.000e+00, 7.500e+02],
         ...,
         [1.000e+00, 3.015e+04],
         [1.000e+00, 3.045e+04],
         [1.000e+00, 3.075e+04]],
 
        [[2.000e+00, 1.500e+02],
         [2.000e+00, 4.500e+02],
         [2.000e+00, 7.500e+02],
         ...,
         [2.000e+00, 3.015e+04],
         [2.000e+00, 3.045e+04],
         [2.000e+00, 3.075e+04]],
 
        ...,
 
        [[9.800e+01, 1.500e+02],
         [9.800e+01, 4.500e+02],
         [9.800e+01, 7.500e+02],
         ...,
         [9.800e+01, 3.015e+04],
         [9.800e+01, 3.045e+04],
         [9.800e+01, 3.075e+04]],
 
        [[9.900e+01, 1.500e+02],
         [9.900e+01, 4.500e+02],
         [9.900e+01, 7.500e+02],
         ...,
         [9.

In [8]:
help(model.run)

Help on method run in module reservoirflow.solutions.analytical.d1p1:

run(nsteps=10, threading=True, vectorize=True, check_MB=True, print_arrays=False, isolver=None) method of reservoirflow.solutions.analytical.d1p1.D1P1 instance
    Solve multiple timesteps.
    
    .. attention::
        This is an abstract method.



In [9]:
model.run(**run_config)

[info] Simulation run started: 100 timesteps.


[step] 1:   0%|          | 0/100 [00:00<?, ?steps/s]


TypeError: D1P1.solve() takes 1 positional argument but 7 were given

In [ ]:
import numpy as np


def logarithmic_scaling(x, min_range, max_range, c=1):
    scaled_value = (
        (np.log(x + c) - np.log(X.min() + c))
        / (np.log(X.max() + c) - np.log(X.min() + c))
    ) * (max_range - min_range) + min_range
    return scaled_value


# Example usage
X = np.array([1, 10, 100, 1000])

min_range = 0  # Replace with your desired minimum range
max_range = 1  # Replace with your desired maximum range

scaled_X = logarithmic_scaling(X, min_range, max_range)

print(scaled_X)


def inverse_logarithmic_scaling(scaled_value, min_range, max_range, c=1):
    original_value = (
        np.exp(
            (
                (scaled_value - min_range)
                * (np.log(X.max() + c) - np.log(X.min() + c))
                / (max_range - min_range)
            )
            + np.log(X.min() + c)
        )
        - c
    )
    return original_value


# Example usage
scaled_X = np.array([0, 0.5, 0.8, 1])  # Replace with your scaled values
min_range = 0  # Replace with your specified minimum range
max_range = 1  # Replace with your specified maximum range

original_X = inverse_logarithmic_scaling(scaled_X, min_range, max_range)

print(original_X)

```{include} /_static/comments_section.md
```